Model typu filtrowanie kolaboracyjne (collaborative filtering).

Wykorzystujemy w tym celu macierz interakcji, którą następnie dekomponujemy na macierze mniejszej wymiarowości z użyciem 
TruncatedSVD - sklearn. 

In [8]:
import pandas as pd
import numpy as np

Przygotowanie macierzy interakcji

In [9]:
sessionsDataPath = '../notebooks/data/v2/sessions.jsonl'
sessionsDF = pd.read_json(sessionsDataPath, lines=True)

df = sessionsDF.drop(columns=["session_id", "timestamp", "event_type", "offered_discount", "purchase_id"])
df["count"] = 1
interactionMatrixDF = pd.pivot_table(df, index="user_id", columns="product_id", values="count", aggfunc=np.sum, fill_value=0)

Dekompozycja utworzonej macierzy na podmacierze ze względu na użytkowników i produkty.

In [10]:
from sklearn.decomposition import TruncatedSVD

#initial hiperparameters
epsilon = 1e-9
latentFactors = 10

#generate item latent features
itemSVD = TruncatedSVD(n_components=latentFactors)
itemFeatures = itemSVD.fit_transform(interactionMatrixDF.transpose()) + epsilon #transpose because items are columns

#generate user latent features
itemSVD = TruncatedSVD(n_components=latentFactors)
itemFeatures = itemSVD.fit_transform(interactionMatrixDF) + epsilon